# Agentic Sales Pipeline

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Load API tokens for our 3rd party APIs

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [4]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [6]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [7]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2024-10-26 06:39:06,099 - 139757045484416 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [8]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [9]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [10]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [11]:
emails = await flow.kickoff()

2024-10-26 06:39:06,140 - 139757045484416 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int



# Agent: Lead Data Specialist
## Thought: Thought: It seems that I'm unable to read the content from the site because it requires JavaScript and cookies, which I cannot enable. I will try to gather the required information from another source in the previous search results to find the company details.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.6sense.com/tech/market-research/clearbit-market-share\"}"
## Tool Output: 
Just a moment...Enable JavaScript and cookies to continue


# Agent: Lead Data Specialist
## Thought: Thought: I need to gather the required company information, but it seems that the websites I have attempted to read are inaccessible due to JavaScript requirements. I will continue searching the previous results to find an alternative source that has the needed information about Clearbit.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.linkedin.com/company/clearbit\"}"
## Tool Output: 
Clearbi



# Agent: Cultural Fit Analyst
## Thought: I need to gather information about Clearbit's cultural values, mission, and overall fit with CrewAI's objectives. This involves researching their publicly stated values and cultural environment to assess alignment.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit cultural values and company mission\"}"
## Tool Output: 

Search results: Title: Defining Clearbit's company values
Link: https://clearbit.com/blog/company-values
Snippet: Values provide a constitution for company culture, a rubric for how you work together. Here's how Clearbit came up with our company values ...
---
Title: Clearbit Mission, Values, Culture & Jobs - Teal
Link: https://www.tealhq.com/company/clearbit
Snippet: Clearbit's core mission is centered around empowering businesses to understand and engage with their customers more effectively through data-driven insights.
---
Title: Mission, Vision & Core Values of Clearbit - Business Model Ca



# Agent: Cultural Fit Analyst
## Thought: Thought: I've gathered detailed information about Clearbit's cultural values and mission. Now, I will read another source to solidify the understanding of their objectives and overall cultural environment before assessing the alignment with CrewAI.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.tealhq.com/company/clearbit\"}"
## Tool Output: 
Clearbit Mission, Values, Culture & Jobs
Log inSign upLog inAI Resume BuilderJob Application Tracker
Tools
AI Resume BuilderMake your resume stand out by quickly customizing it to each application and applying to more jobs.
Job Application TrackerKeep track of your job applications and manage your full job search, all in one place.
Resume Job Description MatchQuickly match and compare your existing resume to the job description of any role for a personalized Match Score.
Resume Bullet Point GeneratorGenerate metric-focused resume bullet points that highlight your big



# Agent: Lead Scorer and Validator
## Thought: I need to gather relevant information about the company Clearbit, specifically regarding its company size, market presence, and cultural fit, to calculate the lead score for João Moura.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Company Clearbit size and market presence information\"}"
## Tool Output: 

Search results: Title: Leverage 100+ business data attributes - Clearbit
Link: https://clearbit.com/attributes
Snippet: Enrich your CRM and database, manage leads efficiently, and enable marketing personalization with over 100 business data points.
---
Title: Clearbit - Market Share, Competitor Insights in Market Research
Link: https://www.6sense.com/tech/market-research/clearbit-market-share
Snippet: The majority of Clearbit's customers for the market-research category fall in the company size of 20 - 49 employees (1,552 companies). You can view a ...
---
Title: Companies using Clearbit and its marketshare -

2024-10-26 06:39:11,073 - 139757045484416 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Scorer and Validator
## Thought: Thought: Since I cannot access the content of the website directly due to restrictions, I will read another relevant page that might provide the needed information regarding Clearbit's company size and market presence.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://enlyft.com/tech/products/clearbit\"}"
## Tool Output: 
Companies using Clearbit and its marketshare
 Platform Overview Account Fit Buyer Personas Buying Signals GenAI Engage Buyer Intent Integrations Propensity Models Browser Extension Data Attributes Solutions Technology Targeting CloudAscent for Microsoft Partners Custom Solutions
 ISV Targeting MSP Targeting High Growth Companies Plans Resources About Enlyft Our customers Careers Contact Us Trust Commitment Sign In LET'S TALK 
Menu
 Platform Plans Solutions Resources About Enlyft Sign In LET'S TALK 
Search field
All Products
›
Marketing Analytics
›
Clearbit
All Products
>
Marketing Analytic

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [12]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0119


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,79354,72670,6684,35


In [13]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,1781,1464,317,3


## Inspecting Results

In [14]:
scores = flow.state["score_crews_results"]

In [15]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,Director of Engineering
Job Title,Director of Engineering
Role Relevance,10
Professional Background,None
Company Name,Clearbit
Industry,Computer Software
Company Size,20
Revenue,None
Market Presence,10
Lead Score,90


## Results

In [16]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Discover how CrewAI can transform your data enrichment processes. Our Multi-
Agent Orchestration Platform is tailored to streamline automation across your
workflows in the Computer Software industry.   Ready to optimize efficiency and
unleash innovation? Click here to schedule a meeting.   Don’t miss out on the
potential to supercharge Clearbit’s operations. Act now and let’s discuss how
our platform can support your ambitious goals.   [Schedule a Meeting](#)


## How Complex Can it Get?

In [17]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [18]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [19]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)